# Hypothesis

Buying when MACD crosses above its signal line and selling when MCAD crosses below its signal line will outperform buy and hold. 

**Steps**
1. Get 2 years of TSLA daily candles
2. Add MACD and MACD Signal
3. Add trade column containing:
    * Buy if: previous rows value is blank; and MCAD > MCAD Signal   
    * Hold if: previous rows value is Buy or Hold; and MCAD > MCAD Signal
    * Sell if: previous rows value is Buy or Hold; and MCAD < MCAD Signal
    * Blank if: previous rows value is Sell or blank; and MCAD < MCAD Signal
4. Add cumulative profit column which is updated on Sell
5. Calculate profit from buy and hold vs total cumulative profit from above strategy
    
**Assumptions**
* Trades cost £10 in both directions (source IG Markets)

## Step 1: Get the candles 
When run on 5DEC21 Yahoo finance only goes back as far as 15MAR21. Downloaded manually as csv file in ~/lab/data. Data from 8OCT21 to 3DEC21. This gives us 2 years worth + some extra days for inital MACD, MACD Signal and RSI calculations.

In [1]:
import pandas as pd
import datetime as dt

# Load from CSV and sort
candles = pd.read_csv('~/data/TSLA.csv')
candles = candles.sort_index()

candles

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-10-08,47.174000,48.787998,46.900002,48.009998,48.009998,43391000
1,2019-10-09,48.264000,49.459999,48.130001,48.905998,48.905998,34472000
2,2019-10-10,49.056000,49.855999,48.316002,48.948002,48.948002,31416500
3,2019-10-11,49.430000,50.216000,49.362000,49.577999,49.577999,42377000
4,2019-10-14,49.580002,51.709999,49.425999,51.391998,51.391998,51025000
...,...,...,...,...,...,...,...
540,2021-11-29,1100.989990,1142.670044,1100.189941,1136.989990,1136.989990,19464500
541,2021-11-30,1144.369995,1168.000000,1118.000000,1144.760010,1144.760010,27092000
542,2021-12-01,1160.699951,1172.839966,1090.760010,1095.000000,1095.000000,22816800
543,2021-12-02,1099.060059,1113.000000,1056.650024,1084.599976,1084.599976,24371600


## Step 2: Add the indicators
Calculate MACD; and MACD Signal:
* MACD from pandas_ta
* MACD Signal using pandas EMA calculation for 9 periods over MACD (ewm.mean)

In [2]:
import pandas_ta as ta

# Copy data first so that we don't change the candles
ta_data = candles.copy()

# Add Indicators
macd = ta_data.ta.macd(close=ta_data['Close'])
ta_data['MACD'] = macd['MACD_12_26_9']
ta_data['MACD_SIGNAL'] = macd['MACDs_12_26_9']

# Remove first 35 rows, 26 of these were only used to calculate the first moving averages and 9 of these were only used 
# to calculate the signal line.
ta_data = ta_data.iloc[35: , :].copy()

ta_data

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,MACD_SIGNAL
35,2019-11-26,67.054001,67.099998,65.419998,65.783997,65.783997,39737000,3.947708,5.438827
36,2019-11-27,66.223999,66.786003,65.713997,66.258003,66.258003,27778000,3.518488,5.054759
37,2019-11-29,66.222000,66.251999,65.500000,65.987999,65.987999,12328000,3.120569,4.667921
38,2019-12-02,65.879997,67.276001,65.737999,66.973999,66.973999,30372500,2.851902,4.304717
39,2019-12-03,66.524002,67.582001,66.438004,67.239998,67.239998,32868500,2.630127,3.969799
...,...,...,...,...,...,...,...,...,...
540,2021-11-29,1100.989990,1142.670044,1100.189941,1136.989990,1136.989990,19464500,40.411584,49.022207
541,2021-11-30,1144.369995,1168.000000,1118.000000,1144.760010,1144.760010,27092000,40.665915,47.350949
542,2021-12-01,1160.699951,1172.839966,1090.760010,1095.000000,1095.000000,22816800,36.432290,45.167217
543,2021-12-02,1099.060059,1113.000000,1056.650024,1084.599976,1084.599976,24371600,31.870532,42.507880


## Step 3: Calculate the trades
* Define function to calculate trade signal. This will need previous rows trade, MACD and MACD Signal. Previous trade will be stored as global as lambda won't have access to previous rows for this calculation. This should return:
    * BUY if: previous trade is 'NONE'; and MCAD > MCAD Signal
    * HOLD if: previous trade is BUY or HOLD; and MCAD > MCAD Signal
    * SELL if: previous trade is BUY or HOLD; and MCAD < MCAD Signal
    * NONE if: previous trade is SELL or 'NONE; and MCAD < MCAD Signal
* Use lamda to apply to dataframe

In [3]:
# Define the calc_trade function
def calc_trade(macd, macd_signal):
    # Access trade global. This will hold the previous trade for each itteration.
    global trade
    
    # Change value of trade depending on previous trade and indicators
    if trade == 'NONE' and macd > macd_signal:
        trade = 'BUY'
    elif trade in ['BUY', 'HOLD'] and macd > macd_signal:
        trade = 'HOLD'
    elif trade in ['BUY', 'HOLD'] and macd < macd_signal:
        trade = 'SELL'
    elif trade in ['NONE', 'SELL'] and macd < macd_signal:
        trade = 'NONE'
            
    return trade

# Set initial value for trade and apply calc_trade to a copy of the dataframe
trade = 'NONE'
trade_data = ta_data.copy()
trade_data['trade'] = trade_data.apply(lambda row : calc_trade(row['MACD'], row['MACD_SIGNAL']), axis = 1)

trade_data.loc[trade_data['trade'] != 'NONE']

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,MACD_SIGNAL,trade
48,2019-12-16,72.510002,76.722000,72.500000,76.300003,76.300003,90871000,2.679756,2.476154,BUY
49,2019-12-17,75.797997,77.099998,75.180000,75.797997,75.797997,42484000,2.926208,2.566165,HOLD
50,2019-12-18,76.125999,79.043999,76.115997,78.629997,78.629997,70605000,3.311865,2.715305,HOLD
51,2019-12-19,79.463997,81.370003,79.300003,80.807999,80.807999,90535500,3.750019,2.922248,HOLD
52,2019-12-20,82.057999,82.599998,80.038002,81.117996,81.117996,73763500,4.075296,3.152857,HOLD
...,...,...,...,...,...,...,...,...,...,...
524,2021-11-04,1234.410034,1243.489990,1217.000000,1229.910034,1229.910034,25397400,111.610961,83.323419,HOLD
525,2021-11-05,1228.000000,1239.869995,1208.000000,1222.089966,1222.089966,21579900,114.203788,89.499492,HOLD
526,2021-11-08,1149.790039,1197.000000,1133.000000,1162.939941,1162.939941,33445700,110.215216,93.642637,HOLD
527,2021-11-09,1173.599976,1174.500000,1011.520020,1023.500000,1023.500000,59105800,94.710839,93.856278,HOLD


## Step 4: Calculate cumulative profit
* Filter to trade rows only.
* Add profit to every SELL row by taking close price - previous rows close price. Previous row will be the corresponding BUY
* Sum the profits for cumulative profit

In [4]:
# Get the trade rows
trades = trade_data.loc[trade_data['trade'].isin(['BUY', 'SELL'])].copy()

# Create a shift column containing the previous rows Close value so that we can access in lambda function to calculate profit
trades['prev_close'] = trades['Close'].shift(1)

# Define method to calculate. Will return close - prev_close - broker_fees if trade_direction is SELL. 
# Otherwise will return None
def calculate_profit(trade_direction, close, prev_close, broker_fees):
    profit = None
    if trade_direction == 'SELL':
        profit = close - prev_close - broker_fees  
        
    return profit

trades['profit_inc_fees'] = trades.apply(lambda row : calculate_profit(row['trade'], row['Close'], row['prev_close'], 20), axis = 1)
trades['profit_excl_fees'] = trades.apply(lambda row : calculate_profit(row['trade'], row['Close'], row['prev_close'], 0), axis = 1)


cumulative_profit_inc_fees = trades['profit_inc_fees'].sum()
cumulative_profit_excl_fees = trades['profit_excl_fees'].sum()

cumulative_profit_excl_fees

818.8059760000001

## Step 5: Compare cumulative profit vs buy and hold
* Compare against buy and hold, buying at first row and selling at last
* Compare against buy and hold, buying at first BUY and selling at last SELL

In [5]:
# Cumulative profit from strategy excluding fees. Buy price is taken as first buy
buy_price = trade_data['Close'].iloc[0]
profit = cumulative_profit_excl_fees
profit_pct = profit / buy_price * 100
print(f"Strategy: Buying and selling using strategy but ignoring broker fees generated ${profit:,.2f}. A return of {profit_pct:.2f}%")

# Cumulative profit from strategy including fees. Buy price is taken as first buy
buy_price = trade_data['Close'].iloc[0]
profit = cumulative_profit_inc_fees
profit_pct = profit / buy_price * 100
print(f"Strategy: Buying and selling using strategy including broker fees generated ${profit:,.2f}. A return of {profit_pct:.2f}%")


# Profit from last row - first row
buy_price = trade_data['Close'].iloc[0]
sell_price = trade_data['Close'].iloc[-1]

profit = sell_price - buy_price - 20  # 20 is broker fees in each direction
profit_pct = profit / buy_price * 100

print(f"First vs Last Row: Buying at ${buy_price:,.2f} and selling at ${sell_price:,.2f} generated ${profit:,.2f}. A return of {profit_pct:.2f}%")

# Profit from last SELL - first BUY
buy_price = trades[trades['trade'] == 'BUY']['Close'].iloc[0]
sell_price = trades[trades['trade'] == 'SELL']['Close'].iloc[-1]

profit = sell_price - buy_price - 20  # 20 is broker fees in each direction
profit_pct = profit / buy_price * 100

print(f"First Buy vs Last Sell: Buying at ${buy_price:,.2f} and selling at ${sell_price:,.2f} generated ${profit:,.2f}. A return of {profit_pct:.2f}%")

Strategy: Buying and selling using strategy but ignoring broker fees generated $818.81. A return of 1244.69%
Strategy: Buying and selling using strategy including broker fees generated $378.81. A return of 575.83%
First vs Last Row: Buying at $65.78 and selling at $1,014.97 generated $929.19. A return of 1412.48%
First Buy vs Last Sell: Buying at $76.30 and selling at $1,067.95 generated $971.65. A return of 1273.46%


## Conclusion
With the data available, buying when MACD crossed above its signal line and selling when MCAD crossed below its signal line underperformed a simple buy and hold, and a MACD timed buy and hold. This was the case both when ignoring broker fees for trading and when accounting for fees. 